In [ ]:
import math
import time
import random
from tkinter import *
from PIL import Image, ImageTk
import numpy as np
#from numba import jit
import mpmath as mpm

In [ ]:
def clamp(x):
    return max(0, min(x, 255))

In [ ]:
def translate(value, leftMin, leftMax, rightMin, rightMax):
    leftSpan = leftMax - leftMin
    rightSpan = (rightMax - rightMin)
    valueScaled = (value - leftMin) / (leftSpan)
    return rightMin + (valueScaled * rightSpan)

In [ ]:
def deepZoomPoint(center_r, center_i, depth=1000):
    v = [];
    xn_r = center_r;
    xn_i = center_i;

    for i in range(depth):
        # pre multiply by two
        re = xn_r + xn_r;
        im = xn_i + xn_i;

        v.append(complex(re,im));

        # make sure our numbers don't get too big
        if (re > 1024 or im > 1024 or re < -1024 or im < -1024):
            return v;

        # calculate next iteration, remember re = 2 * xn_r
        xn_r = xn_r * xn_r - xn_i * xn_i + center_r;
        xn_i = re * xn_i + center_i;

    return v

In [ ]:
#calculate constants at depth
#x is vector containing saved perturbation coefficients
def pt(i, j, w, h, xDelta, yDelta, deepZoom):
    #int window_radius = (size.x < size.y) ? size.x : size.y;
    # find the complex number at the center of this pixel
    d0 = complex(xDelta*(2*i-w)/w,yDelta*(2*j-h)/h)
    dn=(deepZoom[0]+d0)*d0+d0

    maxIteration = len(deepZoom)
    #dn *= deepZoom[0] + dn
    #dn += d0
    # run the iteration loop
    for iteration in range(1,maxIteration):
        zn_size = abs(deepZoom[iteration] * 0.5 + dn)
        # use bailout radius of 256 for smooth coloring.
        if (zn_size > 256):
            break
            
        dn = (deepZoom[iteration] + dn)*dn+d0
        #dn += d0
    if zn_size<2:
        a=0
    else:
        a=math.log(math.log(zn_size,2),2)

    b=int((iteration-a)*10)
    return b if b>0 else 0

In [ ]:
class Mandelbrot():
    def __init__(self, canvasW, canvasH, x=mpm.mpf(0), y=mpm.mpf(0), m=mpm.mpf('1.5'), w=None, h=None, zoomFactor=mpm.mpf('0.5')):
        self.w, self.h = (round(canvasW*0.9), round(canvasH*0.9)) if None in {w, h} else w, h
        self.iterations = 300
        self.xCenter, self.yCenter = x, y
        if canvasW > canvasH:
            self.xDelta = m/(mpm.mpf((canvasH/canvasW)))
            self.yDelta = m
        else:
            self.yDelta = m/(mpm.mpf((canvasW/canvasH)))
            self.xDelta = m
        self.delta = m

        self.xmin = x - self.xDelta
        self.xmax = x + self.xDelta
        self.ymin = y - self.yDelta
        self.ymax = y + self.yDelta
        self.zoomFactor = zoomFactor
        self.yScaleFactor = mpm.mpf(self.h)/mpm.mpf(canvasH)
        self.xScaleFactor = mpm.mpf(self.w)/mpm.mpf(canvasW)
        
    def zoomIn(self, event):
        self.xCenter = translate(mpm.mpmathify(event.y), mpm.mpf(0), mpm.mpmathify(self.w), mpm.mpmathify(self.xmin), mpm.mpmathify(self.xmax))
        self.yCenter = translate(mpm.mpmathify(event.x), mpm.mpf(0), mpm.mpmathify(self.h), mpm.mpmathify(self.ymin), mpm.mpmathify(self.ymax))
        self.xDelta *= self.zoomFactor
        self.yDelta *= self.zoomFactor
        self.delta *= self.zoomFactor
        self.xmax = self.xCenter + self.xDelta
        self.ymax = self.yCenter + self.yDelta
        self.xmin = self.xCenter - self.xDelta
        self.ymin = self.yCenter - self.yDelta
    
    def getPixels(self):
        #self.pixels=jitGetPixels(self.xmin, self.xmax, self.ymin, self.ymax, self.w, self.h, self.iterations)
        #self.pixels = pixels
        #r1 = np.linspace(self.xmin,self.xmax,self.w)
        #r2 = np.linspace(self.ymin,self.ymax,self.h)
        #print(self.h)
        #a = [0 for n in range(self.h)]
        #b = numpy.array(a)
        deepZoom = deepZoomPoint(self.xCenter, self.yCenter)
        pixels = [] #np.empty((self.w, self.h))
        print(self.xmax, self.xmin, self.w)
        print(self.ymax, self.ymin, self.h)
        for i in range(self.w):
            for j in range(self.h):
                pixels.append(pt(i, j, self.w, self.h, self.xDelta, self.yDelta, deepZoom))
        self.pixels = pixels
    
    def getEscapeTime(self, loc):
        #re = translate(x, 0, self.w, self.xmin, self.xmax)
        #im = translate(y, 0, self.h, self.ymax, self.ymin)
        re=cre
        im=cim
        #z, c = complex(re, im), complex(re, im)
        for i in range(1, self.iterations):
            re2 = re*re
            im2 = im*im
            if (re2+im2) > 4:
        #    if abs(z) > 2:
                return (i)
        #    z = z*z + c
            im = 2*re*im+cim
            re = re2-im2+cre
        return (0)

In [ ]:
class Framework(Frame):
    def __init__(self, parent, h, x=mpm.mpf('-.75'), y=mpm.mpf('0'), m=mpm.mpf('1.5')):
        Frame.__init__(self, parent)
        self.parent = parent
        self.parent.title("Mandelbrot")
        self.pack(fill=BOTH, expand=1)
        self.canvas = Canvas(self)
        imgWidth, imgHeight = h, h
        
        if imgWidth > imgHeight:
            ratio = imgHeight/imgWidth
            self.canvasW, self.canvasH = h, round(h*ratio)
        else:
            ratio = imgWidth/imgHeight
            self.canvasW, self.canvasH = round(h*ratio), h
        
        self.background = None
        self.fractal = Mandelbrot(self.canvasW, self.canvasH, x=x, y=y, m=m, w=imgWidth, h=imgHeight)
        self.setPalette()

        #self.img = Image.new('RGB', (self.fractal.w, self.fractal.h), "black")
        self.draw()

        parent.bind("<Button-1>", self.zoomIn)
        #parent.bind("<Button-3>", self.zoomOut)
        #parent.bind("<Control-1>", self.shiftView)
        #parent.bind("<Control-3>", self.changePalette)
        #parent.bind("<Button-2>", self.saveImage)
    
    def zoomIn(self, event):
        self.fractal.zoomIn(event)
        self.draw()
        
    def draw(self):
        print('-' * 20)
        start = time.time()
        self.fractal.getPixels()
        self.getColors()
        self.drawPixels()
        self.canvas.create_image(0, 0, image=self.background, anchor=NW)
        self.canvas.pack(fill=BOTH, expand=1)
        
        print("Process took {} seconds".format(round(time.time()-start, 2)))
        print("Current coordinates (x, y, m): {}, {}, {}".format(self.fractal.xCenter, self.fractal.yCenter, self.fractal.delta))

    def setPalette(self):
        #initialize the palette array, and fill the first value with 0
        palette = np.empty((256,3), dtype='int')
        palette[0,:] = (0, 0, 0)
        redb = 2 * math.pi / (random.randint(0, 128) + 128)
        redc = 256 * random.random()
        greenb = 2 * math.pi / (random.randint(0, 128) + 128)
        greenc = 256 * random.random()
        blueb = 2 * math.pi / (random.randint(0, 128) + 128)
        bluec = 256 * random.random()
        for i in range(1, 256):
            r = clamp(int(256 * (0.5 * math.sin(redb * i + redc) + 0.5)))
            g = clamp(int(256 * (0.5 * math.sin(greenb * i + greenc) + 0.5)))
            b = clamp(int(256 * (0.5 * math.sin(blueb * i + bluec) + 0.5)))
            palette[i,:] = (r, g, b)
        self.palette = palette

    def getColors(self):
        c=0
        pixelColors = np.empty((self.canvasW, self.canvasH, 3), dtype = 'uint8')
        for i in range(self.canvasW):
            for j in range(self.canvasH):
                pixelColors[i,j,:] = list(self.palette[int(self.fractal.pixels[c]%256)])
                c+=1
        self.pixelColors = pixelColors
        
    def drawPixels(self):
        #img = Image.new('RGB', (self.fractal.w, self.fractal.h), "black")
        img = Image.fromarray(self.pixelColors)
        #img.save("pict.png", "PNG", optimize=True)
        self.img = img
        photoimg = ImageTk.PhotoImage(img.resize((self.canvasW, self.canvasH)))
        self.background = photoimg

In [ ]:
master = Tk()
height = round(master.winfo_screenheight()*0.3)
render = Framework(master, height)
master.geometry("{}x{}".format(render.canvasW, render.canvasH))
master.mainloop()

In [ ]:
a=deepZoomPoint(mpm.mpf('-.16'),mpm.mpf('1.0405'))

In [ ]:
pt(185,185,360,360,.026,.026,a)

In [ ]:
c=complex(-.1601,1.0404)
z=c
for i in range(100):
    z=z*z+c
    print(z)
    if abs(z)>2:
        break
print(i)

In [ ]:
math.log(math.log(1.5,2),2)